# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Benisty  ->  M. Benisty  |  ['M. Benisty']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
Arxiv has 52 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2410.13932


extracting tarball to tmp_2410.13932...

 done.


Found 93 bibliographic references in tmp_2410.13932/shadows.bbl.
Retrieving document from  https://arxiv.org/e-print/2410.13939


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem[\protect\citeauthoryear{{Rometsch}, {Ziampras}, {Kley}    {Béthune}}{{Rometsch} et~al.}{2021}]{rometsch-etal-2021}{Rometsch} T.,  {Ziampras} A.,  {Kley} W.,   {Béthune} W.,  2021, \mn@doi  [\aap] {None}, \href {None} {None, None}
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3}).*?href(.*?{(?P<url>http[\S]*)})(?P<rest>.*)
        
  warnings.warn(str(e))


extracting tarball to tmp_2410.13939... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.13932-b31b1b.svg)](https://arxiv.org/abs/2410.13932) | **Spirals, rings, and vortices shaped by shadows in protoplanetary disks: from radiative hydrodynamical simulations to observable signatures**  |
|| A. Ziampras, et al. -- incl., <mark>M. Benisty</mark> |
|*Appeared on*| *2024-10-21*|
|*Comments*| *17 pages, 20 figures, 1 table; submitted to MNRAS; suggestions and comments welcome*|
|**Abstract**|            Numerous protoplanetary disks exhibit shadows in scattered light observations. These shadows are typically cast by misaligned inner disks and are associated with observable structures in the outer disk such as bright arcs and spirals. Investigating the dynamics of the shadowed outer disk is therefore essential in understanding the formation and evolution of these structures. We carry out twodimensional radiation hydrodynamics simulations that include radiative diffusion and dust-gas dynamics to study the formation of substructure in shadowed disks. We find that spiral arms are launched at the edge of each shadow, permeating the entire disk. The local dissipation of these spirals results in an angular momentum flux, opening multiple gaps and leading to a series of concentric, regularly-spaced rings We find that ring formation is favored in weakly turbulent disks where dust growth is taking place. These conditions are met for typical class-II disks, in which bright rings should form well within a fraction of their lifetime (0.1-0.2 Myr). For hotter disks gap opening is more efficient, such that the gap edges quickly collapse into vortices that can appear as bright arcs in continuum emission before decaying into rings or merging into massive, long-lived structures. Synthetic observations show that these structures should be observable in scattered light and millimeter continuum emission, providing a new way to probe the presence of substructure in protoplanetary disks. Our results suggest that the formation of rings and gaps is a common process in shadowed disks, and can explain the rich radial substructure observed in several protoplanetary disks.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.13939-b31b1b.svg)](https://arxiv.org/abs/2410.13939) | **A search for self-lensing binaries with TESS and constraints on their occurrence rate**  |
|| N. Yamaguchi, <mark>K. El-Badry</mark> |
|*Appeared on*| *2024-10-21*|
|*Comments*| *24 pages, 17 figures, submitted to PASP*|
|**Abstract**|            Five self-lensing binaries (SLBs) have been discovered with Kepler light curves. They contain white dwarfs (WDs) in AU-scale orbits that gravitationally lens solar-type companions. Forming SLBs likely requires common envelope evolution when the WD progenitor is an AGB star and has a weakly bound envelope. No SLBs have yet been discovered with data from the Transiting Exoplanet Survey Satellite (TESS), which observes far more stars than Kepler did. Identifying self-lensing in TESS data is made challenging by the fact that TESS only observes most stars for $\sim$25 days at a time, so only a single lensing event will be observed for typical SLBs. TESS's smaller aperture also makes it sensitive only to SLBs a factor of $\sim$100 brighter than those to which Kepler is sensitive. We demonstrate that TESS has nevertheless likely already observed $\sim$4 times more detectable SLBs than Kepler. We describe a search for non-repeating self-lensing signals in TESS light curves and present preliminary candidates for which spectroscopic follow-up is ongoing. We calculate the sensitivity of our search with injection and recovery tests on TESS and Kepler light curves. Based on the 5 SLBs discovered with Kepler light curves, we estimate that $(1.1 \pm 0.6)\%$ of solar-type stars are orbited by WDs with periods of 100-1000 d. This implies a space density of AU-scale WD + main sequence (MS) binaries a factor of 20-100 larger than that of astrometrically-identified WD + MS binaries with orbits in Gaia DR3. We conclude that the Gaia sample is still quite incomplete, mainly because WD + MS binaries can only be unambiguously identified as such for high mass ratios.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2410.13932/./fiducial.png', 'tmp_2410.13932/./fiducial-1D.png', 'tmp_2410.13932/./fiducial-T.png']
copying  tmp_2410.13932/./fiducial.png to _build/html/
copying  tmp_2410.13932/./fiducial-1D.png to _build/html/
copying  tmp_2410.13932/./fiducial-T.png to _build/html/
exported in  _build/html/2410.13932.md
    + _build/html/tmp_2410.13932/./fiducial.png
    + _build/html/tmp_2410.13932/./fiducial-1D.png
    + _build/html/tmp_2410.13932/./fiducial-T.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\tensor}[1]{\overline{\textbf{#1}}}$
$\newcommand{\tensorGR}[1]{\overline{\bm{{#1}}}}$
$\newcommand{\DP}[2]{\frac{\partial{#1}}{\partial{#2}}}$
$\newcommand{\D}[2]{\frac{\text{d}{#1}}{\text{d}{#2}}}$
$\newcommand{\ep}{e_\mathrm{p}}$
$\newcommand{\ap}{a_\mathrm{p}}$
$\newcommand{\G}{\text{G}}$
$\newcommand{\Mstar}{M_\star}$
$\newcommand{\Lstar}{L_\star}$
$\newcommand{\Rp}{R_\mathrm{p}}$
$\newcommand{\Mp}{M_\mathrm{p}}$
$\newcommand{\hp}{h_\mathrm{p}}$
$\newcommand{\Hp}{H_\mathrm{p}}$
$\newcommand{\Tp}{T_\mathrm{p}}$
$\newcommand{\Pp}{P_\mathrm{p}}$
$\newcommand{\Tb}{T_\mathrm{b}}$
$\newcommand{\Mth}{M_\mathrm{th}}$
$\newcommand{\Msun}{\mathrm{M}_\odot}$
$\newcommand{\Lsun}{\mathrm{L}_\odot}$
$\newcommand{\Mjup}{\mathrm{M}_\mathrm{J}}$
$\newcommand{\Rgas}{\mathcal{R}}$
$\newcommand{\cs}{c_\mathrm{s}}$
$\newcommand{\csiso}{c_\mathrm{s,iso}}$
$\newcommand{\csadb}{c_\mathrm{s}^\mathrm{ad}}$
$\newcommand{\OmegaK}{\Omega_\mathrm{K}}$
$\newcommand{\uK}{u_\mathrm{K}}$
$\newcommand{\mean}[1]{\langle{#1} \rangle}$
$\newcommand{\tauR}{\tau_\mathrm{R}}$
$\newcommand{\tauP}{\tau_\mathrm{P}}$
$\newcommand{\tauReff}{\tau_\mathrm{R}^\mathrm{eff}}$
$\newcommand{\tauPeff}{\tau_\mathrm{P}^\mathrm{eff}}$
$\newcommand{\taueff}{\tau_\mathrm{eff}}$
$\newcommand{\kappaR}{\kappa_\mathrm{R}}$
$\newcommand{\kappaP}{\kappa_\mathrm{P}}$
$\newcommand{\cv}{c_\mathrm{v}}$
$\newcommand{\rhomid}{\rho_\mathrm{mid}}$
$\newcommand{\sigmaSB}{\sigma_\mathrm{SB}}$
$\newcommand{\vel}{\bm{u}}$
$\newcommand{\xh}{{x}_\mathrm{h}}$
$\newcommand{\varpih}{{\varpi}_\mathrm{h}}$
$\newcommand{\tcool}{t_\mathrm{cool}}$
$\newcommand{\bcool}{\beta_\mathrm{cool}}$
$\newcommand{\bsurf}{\beta_\mathrm{surf}}$
$\newcommand{\bmid}{\beta_\mathrm{mid}}$
$\newcommand{\btot}{\beta_\mathrm{tot}}$
$\newcommand{\bcoll}{\beta_\mathrm{coll}}$
$\newcommand{\bbuoy}{\beta_\mathrm{buoy}}$
$\newcommand{\bdiff}{\beta_\text{diff}}$
$\newcommand{\bfld}{\beta_\mathrm{FLD}}$
$\newcommand{\Qvisc}{Q_\mathrm{visc}}$
$\newcommand{\Qcool}{Q_\mathrm{cool}}$
$\newcommand{\Qirr}{Q_\mathrm{irr}}$
$\newcommand{\Qrad}{Q_\mathrm{rad}}$
$\newcommand{\Qrelax}{Q_\mathrm{relax}}$
$\newcommand{\Erad}{E_\mathrm{rad}}$
$\newcommand{\aR}{a_\mathrm{R}}$
$\newcommand{\lrad}{l_\mathrm{rad}}$
$\newcommand{\Sigmag}{\Sigma_\mathrm{g}}$
$\newcommand{\Sigmad}{\Sigma_\mathrm{d}}$
$\newcommand{\velg}{\vel_\mathrm{g}}$
$\newcommand{\veld}{\vel_\mathrm{d}}$
$\newcommand{\St}{\mathrm{St}}$
$\newcommand{\Sc}{\mathrm{Sc}}$
$\newcommand{\ad}{a_\mathrm{d}}$
$\newcommand{\brhod}{\bar{\rho}_\mathrm{d}}$
$\newcommand{\sd}{s_\mathrm{d}}$
$\newcommand{\md}{m_\mathrm{d}}$
$\newcommand{\pluto}{\texttt{PLUTO}}$
$\newcommand{\fargo}{{\texttt{FARGO3D}}}$
$\newcommand{\optool}{\texttt{OpTool}}$
$\newcommand{\radmc}{\texttt{RADMC-3D}}$
$\newcommand{\simio}{\texttt{SIMIO-continuum}}$
$\newcommand{\casa}{\texttt{CASA}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Spirals, rings, and vortices shaped by shadows in protoplanetary disks:\\from radiative hydrodynamical simulations to observable signatures

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.13932-b31b1b.svg)](https://arxiv.org/abs/2410.13932)<mark>Appeared on: 2024-10-21</mark> -  _17 pages, 20 figures, 1 table; submitted to MNRAS; suggestions and comments welcome_

</div>
<div id="authors">

A. Ziampras, et al. -- incl., <mark>M. Benisty</mark>

</div>
<div id="abstract">

**Abstract:** Numerous protoplanetary disks exhibit shadows in scattered light observations. These shadows are typically cast by misaligned inner disks and are associated with observable structures in the outer disk such as bright arcs and spirals. Investigating the dynamics of the shadowed outer disk is therefore essential in understanding the formation and evolution of these structures. We carry out twodimensional radiation hydrodynamics simulations that include radiative diffusion and dust--gas dynamics to study the formation of substructure in shadowed disks. We find that spiral arms are launched at the edge of each shadow, permeating the entire disk. The local dissipation of these spirals	results in an	angular momentum flux, opening multiple gaps and leading to a series of concentric, regularly-spaced rings	We find that ring formation is favored in weakly turbulent disks where dust growth is taking place. These conditions are met for typical class-II disks, in which bright rings should form well within a fraction of their lifetime ( $\sim$ 0.1--0.2 Myr). For hotter disks gap opening is more efficient, such that the gap edges quickly collapse into vortices that can appear as bright arcs in continuum emission before decaying into rings or merging into massive, long-lived structures. Synthetic observations show that these structures should be observable in scattered light and millimeter continuum emission, providing a new way to probe the presence of substructure in protoplanetary disks. Our results suggest that the formation of rings and gaps is a common process in shadowed disks, and can explain the rich radial substructure observed in several protoplanetary disks.

</div>

<div id="div_fig1">

<img src="tmp_2410.13932/./fiducial.png" alt="Fig17" width="100%"/>

**Figure 17. -** The perturbed gas temperature (left) and gas surface density (middle) as well as the dust surface density (right) after $1000 P_0$. Two dark wedges are visible in the temperature structure, corresponding to the attenuated heating due to the "shadows" cast by an inner disk. A series of concentric rings is visible in all quantities, forming clearly visible features in the dust surface density. The shadow edges are marked with dashed lines in the left panel. The disk rotates counterclockwise. (*fig:fiducial*)

</div>
<div id="div_fig2">

<img src="tmp_2410.13932/./fiducial-1D.png" alt="Fig5" width="100%"/>

**Figure 5. -** Azimuthally averaged profiles of the perturbed gas and dust surface densities (panels *a* and *b*), the angular momentum flux $F_J$(panel *c*), and the cooling timescale $\beta$(panel *d*) as a function of radius and for several snapshots. The AMF is approximately constant over time, leading to deeper gaps and sharper features as the disk evolves. The disk is roughly separated into a featureless inner region and a ringed outer region at $R$\ap$prox 24$ au, which corresponds to $\beta$\ap$prox\beta_\text{crit}$(see Eq. \eqref{eq:bcrit}). The AMF is arbitrarily normalized to $F_{J,0}=10^{-10} $\Msun$ \text{au}^2/\text{yr}^2$. (*fig:fiducial-1D*)

</div>
<div id="div_fig3">

<img src="tmp_2410.13932/./fiducial-T.png" alt="Fig4" width="100%"/>

**Figure 4. -** Azimuthal profile of the perturbed gas temperature at different radii at $t=1000 P_0$. The temperature perturbation is not symmetric about the shadow exits, and becomes smaller in the slowly-cooling inner disk. Shaded regions mark the two shadows. Gas moves to the right. (*fig:fiducial-T*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.13932"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

223  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
